## API DOC WHAT I NEED

**General info**:
SUMMONER/V4
- Summoner name
- Summoner level 

**League**:
CHAMPION-MASTERY-V4
- Champ masteries level
- LastplayTime
LEAGUE/V4
- queueType
- tier
- rank
- leaguePoints
- wins
- losses
- inactive
MATCH/V5
- all match data

**What i want to visualize**:
- ✅ Win percentage overall and per role
- ✅ last 30 days/games wins/losses, y bar game time, the color of the bar red or green based on win or loss. Inside the bar the champ name you played that game, the x line the games and dates. A line trough the barplot indicating KDA
- ✅ last 30 days all played champs with their winrate and KDA
- ✅ Items the player build per champ over the last 30 days and the match outcome. To see what items and order worked best.
- ✅ Teammate Synergy Radar Chart:
Create a radar chart that visualizes the synergy between the player and their most frequent teammates. Axes can represent metrics like win rate when playing together, average KDA in shared games, and objective control.

**Data i want for my PowerBi**:
- excisting funciton for summoner data
- excisting function for winrate overall and per role
- excisting function for synergy per top 3 user champions
- KDA and WR per champion
- General data from matches, so we load data from ~50 matches for each person that runs the code, and we load for each match a row in the database.


#### <ins>Table structure<ins>
- Match-Data-Table: MatchID, SummonerDataID, champion played, game duration, KDA, baronkills, dragonkills, wards placed, wards killed, vision score, turret kills, total minions killed, Win/Loss
-✅ Summoner-Data_table: ID, SummonerName, Summoner Level, queuetype
- Summoner-data-ID: ID, summonerName, queuetype
- ✅Winrate-Table: OverallWR, TopWR, JungleWR, MidWR, ADCWR, SupportWR
- ✅SynergyWR-Table: SummonerDataID, SynergyChampName, Column for every league champion, Fill in rows with the Synergy champ, and the WinRate for the top 3 champs of the summoner
- ✅SynergyKDA-Table: SummonerDataID, SynergyChampName, Column for every league champion, Fill in rows with the Synergy champ, and the KDA for the top 3 champs of the summoner

**Note**
- Create the tables beforehand, and make the values be put into the table.

In [12]:
import requests
import matplotlib.pyplot as plt
import time
from lol_data import get_summoner_lol_data
from config import API_KEY


def get_summoner_lol_data(api_key, summoner_name, region):
    lol_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}'
    headers = {'X-Riot-Token': api_key}

    response = requests.get(lol_url, headers=headers)

    if response.status_code == 200:
        summoner_data = response.json()
        return summoner_data
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

def get_match_ids(api_key, puuid, region, queue_type=None, start_time=None, end_time=None, count=50):
    region_mappings = {
        'na1': 'americas',
        'br1': 'americas',
        'lan': 'americas',
        'las': 'americas',
        'kr': 'asia',
        'jp': 'asia',
        'eune': 'europe',
        'euw1': 'europe',
        'tr1': 'europe',
        'ru': 'europe',
        'oce': 'sea',
        'ph2': 'sea',
        'sg2': 'sea',
        'th2': 'sea',
        'tw2': 'sea',
        'vn2': 'sea',
    }

    queue_type_mappings = {
        'ranked_solo_duo': 420,
        'ranked_flex': 440,
        'blind_pick': 430,
        'draft_pick': 400,
        'clash': 700
    }

    region = region.lower()
    routing_value = region_mappings.get(region, region)

    # Convert user-friendly queue type to numeric queue ID
    queue_id = queue_type_mappings.get(queue_type.lower())

    if queue_id is None:
        print("Invalid queue type. Please enter a valid queue type: ranked_solo_duo, ranked_flex, blind_pick, draft_pick, clash")
        return None

    matchlist_url = f'https://{routing_value}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
    headers = {'X-Riot-Token': api_key}

    params = {'startTime': start_time, 'endTime': end_time, 'count': count, 'queue': queue_id}

    try:
        response = requests.get(matchlist_url, headers=headers, params=params)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching match IDs: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Exception during match ID retrieval: {e}")
        return None


def get_match_data(api_key, match_id, region):
    routing_value = get_routing_value(region)

    match_url = f'https://{routing_value}.api.riotgames.com/lol/match/v5/matches/{match_id}'

    headers = {'X-Riot-Token': api_key}

    try:
        response = requests.get(match_url, headers=headers)

        if response.status_code == 200:
            match_data = response.json()
            if 'info' in match_data and 'gameMode' in match_data['info'] and match_data['info']['gameMode'].lower() == 'classic':
                # Extract champion info from the match data and add it to each participant
                for participant in match_data['info']['participants']:
                    champion_id = participant.get('championId')
                    if champion_id:
                        champion_info = {"id": champion_id, "name": participant.get("championName", "Unknown")}
                        participant['championInfo'] = champion_info

                return match_data
            else:
                print(f"Skipping match ID {match_id} as it is not a classic game.")
                return None
        else:
            print(f"Error fetching match data for match ID {match_id}: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Exception during match data retrieval: {e}")
        return None


def calculate_win_percentage_per_role(matches, summoner_puuid):
    # Filter out None values from matches
    valid_matches = [match for match in matches if match is not None]

    # Check if there are valid matches to calculate win percentage
    if valid_matches:
        overall_wins = 0
        overall_matches_with_wins = 0

        role_wins = {role: 0 for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]}
        role_matches_with_wins = {role: 0 for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]}

        for match in valid_matches:
            if 'info' in match and 'participants' in match['info']:
                for participant in match['info']['participants']:
                    champion_info = participant.get('championInfo', {})
                    champion_name = champion_info.get('name', 'Unknown')
                    if 'individualPosition' in participant:
                        role = participant['individualPosition']
                        # Check if the role is valid
                        if role in role_wins:
                            # Check if the summoner is the participant and if they won
                            if participant['puuid'] == summoner_puuid:
                                overall_matches_with_wins += 1
                                role_matches_with_wins[role] += 1
                                if participant.get('win', False):
                                    overall_wins += 1
                                    role_wins[role] += 1

        overall_win_percentage = (overall_wins / overall_matches_with_wins) * 100 if overall_matches_with_wins > 0 else 0

        win_percentages = {"Overall": {"Win Percentage": overall_win_percentage, "Total Matches": overall_matches_with_wins}}

        for role in ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]:
            role_win_percentage = (role_wins[role] / role_matches_with_wins[role]) * 100 if role_matches_with_wins[role] > 0 else 0
            win_percentages[role.capitalize()] = {"Win Percentage": role_win_percentage, "Total Matches": role_matches_with_wins[role]}

        return win_percentages
    else:
        print("No valid match data available.")

def get_routing_value(region):
    region_mappings = {
        'na1': 'americas',
        'br1': 'americas',
        'lan': 'americas',
        'las': 'americas',
        'kr': 'asia',
        'jp': 'asia',
        'eune': 'europe',
        'euw1': 'europe',
        'tr1': 'europe',
        'ru': 'europe',
        'oce': 'sea',
        'ph2': 'sea',
        'sg2': 'sea',
        'th2': 'sea',
        'tw2': 'sea',
        'vn2': 'sea',
    }

    return region_mappings.get(region.lower(), region.lower())



def get_item_name(item_id):
    item_data_url = "https://ddragon.leagueoflegends.com/cdn/13.24.1/data/en_US/item.json"
    response = requests.get(item_data_url)
    item_data = json.loads(response.text)

    if 'data' in item_data and str(item_id) in item_data['data']:
        return item_data['data'][str(item_id)]['name']
    else:
        return f"Unknown Item {item_id}"
    


def calculate_champion_synergy(matches, summoner_puuid, top_champions):
    champion_synergy = {}

    for match in matches:
        if 'info' in match and 'participants' in match['info']:
            user_champion = None
            teammates = []

            for participant in match['info']['participants']:
                if participant['puuid'] == summoner_puuid:
                    user_champion = participant['championInfo']['name']
                else:
                    teammates.append({
                        'champion': participant['championInfo']['name'],
                        'win': participant.get('win', False),
                        'kills': participant['kills'],
                        'deaths': participant['deaths'],
                        'assists': participant['assists']
                    })

            if user_champion in top_champions:
                if user_champion not in champion_synergy:
                    champion_synergy[user_champion] = {}

                for teammate in teammates:
                    teammate_champion = teammate['champion']
                    if teammate_champion not in champion_synergy[user_champion]:
                        champion_synergy[user_champion][teammate_champion] = {'kda': [], 'winrate': [], 'games_played': 0}

                    # Calculate KDA synergy
                    if teammate['deaths'] > 0:
                        kda = (teammate['kills'] + teammate['assists']) / teammate['deaths']
                        champion_synergy[user_champion][teammate_champion]['kda'].append(kda)

                    # Calculate winrate synergy
                    champion_synergy[user_champion][teammate_champion]['winrate'].append(teammate['win'])

                    # Update games played count
                    champion_synergy[user_champion][teammate_champion]['games_played'] += 1

    return champion_synergy


def get_top_champions(matches, summoner_puuid, num_top_champions=3):
    champion_played_count = {}

    for match in matches:
        if 'info' in match and 'participants' in match['info']:
            for participant in match['info']['participants']:
                if participant['puuid'] == summoner_puuid:
                    champion_name = participant['championInfo']['name']
                    champion_played_count[champion_name] = champion_played_count.get(champion_name, 0) + 1

    # Sort champions based on the number of times played
    sorted_champions = sorted(champion_played_count.items(), key=lambda x: x[1], reverse=True)

    # Get the top N champions
    top_champions = [champion[0] for champion in sorted_champions[:num_top_champions]]

    return top_champions



summoner_name = input("Enter Summoner Name: ")
region = input("Enter Region (e.g., na1, euw1): ")
queue_type = input("Enter Queue Type (e.g., ranked_solo_duo): ")

summoner_data = get_summoner_lol_data(API_KEY, summoner_name, region)
match_ids = get_match_ids(API_KEY, summoner_data['puuid'], region, queue_type, count=50)
if summoner_data:
    print(f"Summoner name: {summoner_data['name']}")
    print(f"Summoner Level: {summoner_data['summonerLevel']}")
    print(f"Below is the data analyses for {summoner_name} in {queue_type}:")
    match_ids = get_match_ids(API_KEY, summoner_data['puuid'], region, queue_type)
    if match_ids:
        matches = []
        for match_id in match_ids:
            match_data = get_match_data(API_KEY, match_id, region)
            if match_data:
                matches.append(match_data)
            else:
                print(f"Error fetching match data for match ID {match_id}")

import pyodbc
# from config import API_KEY  # Importeer je API_KEY uit het bestand config.py
# from lol_data import calculate_win_percentage_per_role

import hashlib

def generate_unique_id(summoner_name, queue_type):
    # Hash the combination of summoner name and queue type to create a unique ID
    unique_id = hashlib.md5(f"{summoner_name}-{queue_type}".encode()).hexdigest()
    return unique_id

# Verbinding maken met de database
try:
    conn = pyodbc.connect("Driver={ODBC Driver 18 for SQL Server};Server=tcp:jarvis-cloud-verzamelen-joram.database.windows.net,1433;Database=BitAcademyDB;Uid=152791@student.horizoncollege.nl;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive")
    cursor = conn.cursor()
except pyodbc.Error as e:
    print(f"Fout bij het verbinden met de database: {e}")

win_percentages = calculate_win_percentage_per_role(matches, summoner_data['puuid'])

unique_id = generate_unique_id(summoner_name, queue_type)

try:
    cursor.execute("""
        MERGE INTO Winrate_Table AS Target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?)) AS Source (UniqueID, Overall, TopLane, Jungle, MidLane, ADC, Support)
        ON Target.UniqueID = Source.UniqueID
        WHEN MATCHED THEN
            UPDATE SET
                Overall = Source.Overall,
                TopLane = Source.TopLane,
                Jungle = Source.Jungle,
                MidLane = Source.MidLane,
                ADC = Source.ADC,
                Support = Source.Support
        WHEN NOT MATCHED THEN
            INSERT (UniqueID, Overall, TopLane, Jungle, MidLane, ADC, Support)
            VALUES (Source.UniqueID, Source.Overall, Source.TopLane, Source.Jungle, Source.MidLane, Source.ADC, Source.Support);
    """,
    (unique_id,
    win_percentages['Overall']['Win Percentage'],
    win_percentages['Top']['Win Percentage'],
    win_percentages['Jungle']['Win Percentage'],
    win_percentages['Middle']['Win Percentage'],
    win_percentages['Bottom']['Win Percentage'],
    win_percentages['Utility']['Win Percentage']))

    conn.commit()
    print("Winrates toegevoegd aan de database!")
except pyodbc.Error as e:
    print(f"Fout bij het invoegen van winrates in de database: {e}")
finally:
    cursor.close()
    conn.close()



Summoner name: 420ShadesOfGreen
Summoner Level: 833
Below is the data analyses for 420shadesofgreen in ranked_solo_duo:
57.99999999999999
Winrates toegevoegd aan de database!


In [10]:
import pyodbc

# Connecting to database
try:
    conn = pyodbc.connect("Driver={ODBC Driver 18 for SQL Server};Server=tcp:jarvis-cloud-verzamelen-joram.database.windows.net,1433;Database=BitAcademyDB;Uid=152791@student.horizoncollege.nl;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive")
    cursor = conn.cursor()

except pyodbc.Error as e:
    print(f"Fout bij het verbinden met de database: {e}")
